In [2]:
!nvidia-smi

Sat Jul 11 23:46:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import pandas as pd

In [68]:
data= pd.read_csv("news.csv")

In [69]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [70]:
data= data.drop(["Unnamed: 0", "title"], axis=1)

In [71]:
data

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL
...,...,...
6330,The State Department told the Republican Natio...,REAL
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,"ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [72]:
# Getting stopwords to remove
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop= stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
data["text"]= data["text"].apply(lambda x: " ".join(x.lower() for x in x.split() if x not in stop))

In [74]:
data["text"]= data["text"].str.replace(r'[^\w\s]', " ")

In [11]:
!pip install bert-for-tf2

     |████████████████████████████████| 40kB 2.5MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=e320961622f28d48a8ca0154f7ee1f365c24582bb96078020e06d563fc7d42a7
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=f253a785ec8273a2e56474316506ba93938d43f746305cdc01cfa57f8328d07c
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=27b2e6ed4024eb0e9b125fcdfc4251ab1b47b777669789a0dae085f7daa3ca15
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [12]:
#Getting the bert model
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2020-07-11 23:51:33--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.142.128, 2607:f8b0:400e:c07::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   119MB/s    in 3.3s    

2020-07-11 23:51:37 (119 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.j

In [13]:
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from bert.tokenization.bert_tokenization import FullTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bert
from bert import BertModelLayer
import tensorflow as tf
import json
import os
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights

In [14]:
# paths to BERT "config" and "ckpt" file
bert_config_file= "/content/uncased_L-12_H-768_A-12/bert_config.json"
max_seq_len= 900
bert_ckpt_file= "/content/uncased_L-12_H-768_A-12/bert_model.ckpt"

In [15]:
BERT_VOCAB= "/content/uncased_L-12_H-768_A-12/vocab.txt"
tokenizer= FullTokenizer(BERT_VOCAB) #initialization of tokenizer

In [75]:
data["text"]= data["text"].apply(lambda x: tokenizer.tokenize(x))

In [76]:
data["text"]= data["text"].apply(lambda x: ["[CLS]"]+x+["[SEP]"])

In [77]:
data["text"]= data["text"].apply(lambda x: tokenizer.convert_tokens_to_ids(x))

In [78]:
x= data["text"]
y= data["label"]

In [79]:
x= pad_sequences(x, padding= "post", maxlen= 900, truncating= "post")

In [80]:
from sklearn.preprocessing import LabelEncoder
encoder=  LabelEncoder()
y= encoder.fit_transform(y)

In [81]:
y

array([0, 0, 1, ..., 0, 1, 1])

In [84]:
# function for creating the model
def create_model(max_seq_len, bert_ckpt_file):
  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = None
      bert = BertModelLayer.from_params(bert_params, name="bert")
  input_ids = tf.keras.layers.Input(
    shape=(900, ),
    dtype='int32',
    name="input_ids"
  )
  bert_output = bert(input_ids)
  print("bert shape", bert_output.shape)
  cls_out = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
  cls_out = tf.keras.layers.Dropout(0.5)(cls_out)
  logits = tf.keras.layers.Dense(units=768, activation="tanh")(cls_out)
  logits = tf.keras.layers.Dropout(0.5)(logits)
  logits = tf.keras.layers.Dense(
    units=2,
    activation="softmax"
  )(logits)
  model = tf.keras.Model(inputs=input_ids, outputs=logits)
  model.build(input_shape=(None, max_seq_len))
  load_stock_weights(bert, bert_ckpt_file)
  return model

In [85]:
model= create_model(max_seq_len, bert_ckpt_file)

bert shape (None, 900, 768)
loader: Skipping weight:[bert_4/embeddings/position_embeddings/embeddings:0] as the weight shape:[(900, 768)] is not compatible with the checkpoint:[bert/embeddings/position_embeddings] shape:(512, 768)
Done loading 195 BERT weights from: /content/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7f3f00f5acc0> (prefix:bert_4). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [1]
Unused weights from checkpoint: 
	bert/embeddings/position_embeddings
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [86]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 900)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 900, 768)          109188096 
_________________________________________________________________
lambda_3 (Lambda)            (None, 768)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 768)               590592    
_________________________________________________________________
dropout_7 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 1538

In [87]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(1e-5),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [89]:
history = model.fit(
  x=x, y=y,
  validation_split=0.2,
  batch_size=4,
  shuffle=True,
  epochs=3)

Epoch 1/3
1267/1267 [==============================] - 1029s 812ms/step - loss: 0.4668 - acc: 0.8437 - val_loss: 0.4738 - val_acc: 0.8374
Epoch 2/3
1267/1267 [==============================] - 1029s 812ms/step - loss: 0.4376 - acc: 0.8729 - val_loss: 0.4292 - val_acc: 0.8816
Epoch 3/3
1267/1267 [==============================] - 1030s 813ms/step - loss: 0.4341 - acc: 0.8779 - val_loss: 0.4266 - val_acc: 0.8871


In [90]:
model.save("model.h5")